In [1]:
# ============================================================
# CBIS-DDSM (Kaggle) → torchvision (EfficientNetV2-S)
# ROI tabanlı benign/malign sınıflandırma (Colab, tek parça)
# Çıktılar: /content/drive/MyDrive/breast_cancer
# Albumentations / OpenCV / timm YOK → torchvision.transforms + PIL
# ============================================================

# ---- 0) Minimal kurulum ----
!pip -q install --upgrade pip setuptools wheel kaggle scikit-learn==1.6.1 torchmetrics==1.4.0.post0

import os, json, time, random, math, shutil, zipfile, tarfile, re
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.metrics import (roc_auc_score, accuracy_score, f1_score,
                             confusion_matrix, classification_report,
                             ConfusionMatrixDisplay, roc_curve,
                             precision_recall_curve, average_precision_score)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [4]:
# ---- 2) Google Drive mount (sağlam) ----
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive', force_remount=True)

OUTPUT_DIR = Path('/content/drive/MyDrive/breast_cancer')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("Çıktılar:", OUTPUT_DIR)


Mounted at /content/drive
Çıktılar: /content/drive/MyDrive/breast_cancer


In [5]:
# ---- 3) Kaggle kimliği ve veri indirme ----
from google.colab import files
KAGGLE_DIR = Path('/root/.kaggle'); KAGGLE_DIR.mkdir(parents=True, exist_ok=True)
if not (KAGGLE_DIR/'kaggle.json').exists():
    print("Lütfen bilgisayarındaki kaggle.json dosyasını seç.")
    up = files.upload()
    if 'kaggle.json' not in up: raise RuntimeError("kaggle.json gerekli.")
    with open(KAGGLE_DIR/'kaggle.json', 'wb') as f: f.write(up['kaggle.json'])
!chmod 600 /root/.kaggle/kaggle.json

DATA_ROOT = Path('/content/cbis_ddsm'); DATA_ROOT.mkdir(parents=True, exist_ok=True)
!kaggle datasets download -d awsaf49/cbis-ddsm-breast-cancer-image-dataset -p {DATA_ROOT} -q
for z in DATA_ROOT.glob('*.zip'):
    shutil.unpack_archive(str(z), extract_dir=str(DATA_ROOT))
    z.unlink()

Lütfen bilgisayarındaki kaggle.json dosyasını seç.


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset
License(s): CC-BY-SA-3.0


In [6]:
# ---- 4) İç içe arşivleri aç & envanter ----
def unpack_all_archives(root: Path, max_pass=2):
    for _ in range(max_pass):
        changed = False
        for p in list(root.rglob('*')):
            if not p.is_file(): continue
            low = p.name.lower()
            try:
                if low.endswith('.zip'):
                    with zipfile.ZipFile(p, 'r') as z: z.extractall(p.parent)
                    p.unlink(); changed = True
                elif low.endswith(('.tar', '.tar.gz', '.tgz')):
                    with tarfile.open(p, 'r:*') as t: t.extractall(p.parent)
                    p.unlink(); changed = True
            except Exception as e:
                print("Arşiv açılamadı:", p, "->", e)
        if not changed: break

unpack_all_archives(DATA_ROOT)

all_files = [p for p in DATA_ROOT.rglob('*') if p.is_file()]
ext_count = Counter([p.suffix.lower() for p in all_files])
print(f"[INFO] Toplam dosya: {len(all_files)}")
print("[INFO] Uzantı sayıları:", dict(ext_count))

[INFO] Toplam dosya: 10243
[INFO] Uzantı sayıları: {'.csv': 6, '.jpg': 10237}


In [13]:
# ===================== 5. KISIM (GENEL ÇÖZÜM – DÜZELTİLMİŞ) =====================
from pathlib import Path
import pandas as pd, numpy as np, re
from collections import Counter
from sklearn.model_selection import StratifiedGroupKFold, train_test_split

DATA_ROOT = Path('/content/cbis_ddsm')
assert DATA_ROOT.exists(), f"{DATA_ROOT} yok; indirme/açma kısmını çalıştır."

# 1) Klasördeki TÜM .jpg dosyaları
jpgs = sorted([p for p in DATA_ROOT.rglob('*.jpg')])
jpg_names = set(p.name for p in jpgs)
print(f"[INFO] JPG sayısı: {len(jpgs)}")
print("[INFO] Örnek JPG'ler:", [p.name for p in jpgs[:10]])

# 2) CSV’leri topla (mass/calc + meta/dicom_info dahil)
wanted = {
    'mass_case_description_train_set.csv',
    'mass_case_description_test_set.csv',
    'calc_case_description_train_set.csv',
    'calc_case_description_test_set.csv',
    'meta.csv',
    'dicom_info.csv'
}
csvs = []
for w in wanted:
    csvs.extend(list(DATA_ROOT.rglob(w)))
# yedek: kökteki tüm csv'ler
if not csvs:
    csvs = sorted(DATA_ROOT.rglob('*.csv'))
print("[INFO] Kullanılacak CSV'ler:", [c.name for c in csvs])

# 3) Kolon seçim yardımcıları
def pick_label_col(cols_lower_map):
    # pathology > label > class
    if 'pathology' in cols_lower_map: return cols_lower_map['pathology']
    for k in ('label','class'):
        if k in cols_lower_map: return cols_lower_map[k]
    # fallback
    for k in cols_lower_map:
        if 'patholog' in k: return cols_lower_map[k]
    return None

def normalize_label(v):
    if v is None: return None
    s = str(v).strip().lower()
    if 'benign' in s or s in ('b','0'): return 'benign'
    if 'malig'  in s or s in ('m','1'): return 'malignant'
    return None

def score_filename_column(series):
    """Serideki değerlerin base name'i jpg_names içinde kaç tane eşleşiyor?"""
    try:
        vals = series.astype(str)
    except Exception:
        return 0
    hit = 0
    for v in vals:
        if not v or v == 'nan':
            continue
        base = Path(str(v)).name  # '.../foo.jpg' -> 'foo.jpg'
        if base in jpg_names:
            hit += 1
    return hit

# 4) CSV’lerden direkt .jpg adı yakalamaya çalış
pairs = []  # (jpg_path, label)

for csv_path in csvs:
    # Birkaç kodlama ayrıntısı için esnek okuma denemeleri
    dfc = None
    for kwargs in ({}, {"sep":";"}, {"encoding":"latin1"}, {"sep":";","encoding":"latin1"}):
        try:
            dfc = pd.read_csv(csv_path, **kwargs)
            break
        except Exception:
            continue
    if dfc is None or dfc.empty:
        print(f"[WARN] CSV okunamadı/boş: {csv_path.name}")
        continue

    cols_map = {c.lower(): c for c in dfc.columns}
    ycol = pick_label_col(cols_map)

    # Tüm kolonları gezip file name olma skorunu hesapla
    col_hits = []
    for c in dfc.columns:
        h = score_filename_column(dfc[c])
        if h > 0:
            col_hits.append((h, c))
    col_hits.sort(reverse=True)
    if col_hits:
        best_hits = col_hits[0][0]
        best_col  = col_hits[0][1]
        print(f"[INFO] {csv_path.name} içinde '{best_col}' kolonu {best_hits} adet jpg ile örtüşüyor.")

        # Bu kolon üzerinden etiketle
        if ycol is not None:
            mapped_here = 0
            for _, r in dfc.iterrows():
                base = Path(str(r.get(best_col))).name
                if base in jpg_names:
                    lb = normalize_label(r.get(ycol))
                    if lb is None:
                        continue
                    # jpg tam yolunu bul (aynı ad birden fazla yerdeyse ilkini al)
                    jp = next((p for p in jpgs if p.name == base), None)
                    if jp is not None:
                        pairs.append((jp, lb))
                        mapped_here += 1
            print(f"[INFO] CSV eşleme (doğrudan jpg adı): {csv_path.name} -> {mapped_here}")
    else:
        print(f"[INFO] {csv_path.name} içinde doğrudan jpg adı bulunamadı.")

# 5) Eğer CSV’den hâlâ kimliklendiremediysek, dosya adı deseninden etiket çıkar
if not pairs:
    print("\n[WARN] CSV'den doğrudan jpg adı bulunamadı/etiketlenemedi. Dosya adı desenine bakılıyor…")
    # Örn: '1-xxxx.jpg' ve '2-xxxx.jpg' → 1=benign, 2=malignant (veya tersi).
    prefix_counts = Counter([p.name.split('-')[0] if '-' in p.name else '' for p in jpgs])
    print("[INFO] Dosya adı prefix dağılımı:", dict(prefix_counts))
    if set(prefix_counts.keys()) <= {'1','2',''} and prefix_counts.get('1',0) > 0 and prefix_counts.get('2',0) > 0:
        prefix_to_label = {'1':'benign', '2':'malignant'}   # gerekirse tersine çevir
        pairs = []
        for p in jpgs:
            pref = p.name.split('-')[0] if '-' in p.name else ''
            lb = prefix_to_label.get(pref)
            if lb is not None:
                pairs.append((p, lb))
        print(f"[INFO] Dosya adı deseniyle etiketlenen örnek: {len(pairs)}")
    else:
        raise RuntimeError("Ne CSV’de .jpg adı, ne de öngörülebilir dosya adı deseni bulundu. Eşleme yapılamadı.")

# 6) DataFrame ve split
# aynı path tekrar ederse ilkini tut
uniq = {}
for p, lb in pairs:
    uniq[str(p)] = lb
rows = [(Path(k), v) for k,v in uniq.items()]

df = pd.DataFrame(rows, columns=['path','label'])
df['label_id'] = (df['label']=='malignant').astype(np.float32)
df = df.drop_duplicates(subset=['path']).reset_index(drop=True)

print("\n[OK] Etiket dağılımı:")
print(df['label'].value_counts(dropna=False))
print("Toplam örnek:", len(df))

def guess_group(p):
    p = Path(p)
    parts = p.parts
    return '/'.join(parts[-3:-1]) if len(parts) >= 3 else str(p.parent)

df['group'] = df['path'].apply(guess_group)
df = df.sample(frac=1.0, random_state=123).reset_index(drop=True)

try:
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    idx_tr, idx_va = next(sgkf.split(df, y=df['label_id'], groups=df['group']))
    df_train, df_val = df.iloc[idx_tr].reset_index(drop=True), df.iloc[idx_va].reset_index(drop=True)
except Exception as e:
    print("[WARN] SGKF olmadı, stratified split:", e)
    df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label_id'])
    df_train = df_train.reset_index(drop=True)
    df_val   = df_val.reset_index(drop=True)

print(f"[OK] Split → Train: {len(df_train)}  Val: {len(df_val)}")
# =================== 5. KISIM SONU ===================


[INFO] JPG sayısı: 10237
[INFO] Örnek JPG'ler: ['1-263.jpg', '2-241.jpg', '1-126.jpg', '1-231.jpg', '1-111.jpg', '1-031.jpg', '2-011.jpg', '1-108.jpg', '1-031.jpg', '1-075.jpg']
[INFO] Kullanılacak CSV'ler: ['mass_case_description_test_set.csv', 'dicom_info.csv', 'mass_case_description_train_set.csv', 'calc_case_description_train_set.csv', 'calc_case_description_test_set.csv', 'meta.csv']
[INFO] mass_case_description_test_set.csv içinde doğrudan jpg adı bulunamadı.
[INFO] dicom_info.csv içinde 'image_path' kolonu 10237 adet jpg ile örtüşüyor.
[INFO] mass_case_description_train_set.csv içinde doğrudan jpg adı bulunamadı.
[INFO] calc_case_description_train_set.csv içinde doğrudan jpg adı bulunamadı.
[INFO] calc_case_description_test_set.csv içinde doğrudan jpg adı bulunamadı.
[INFO] meta.csv içinde doğrudan jpg adı bulunamadı.

[WARN] CSV'den doğrudan jpg adı bulunamadı/etiketlenemedi. Dosya adı desenine bakılıyor…
[INFO] Dosya adı prefix dağılımı: {'1': 6774, '2': 3463}
[INFO] Dosya adı

In [14]:
# =========================
# ==== 6) DATASET & TFMS ==
# (Senin istediğin gibi AYNI blok)
# =========================
IMG_SIZE = 512
train_tfms = T.Compose([
    T.Resize(IMG_SIZE, antialias=True),
    T.CenterCrop(IMG_SIZE),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ToTensor(),
    T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])
val_tfms = T.Compose([
    T.Resize(IMG_SIZE, antialias=True),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

class MammoDataset(Dataset):
    def __init__(self, frame: pd.DataFrame, tfms):
        self.frame = frame.reset_index(drop=True)
        self.tfms = tfms
    def __len__(self): return len(self.frame)
    def __getitem__(self, idx):
        row = self.frame.iloc[idx]
        img = Image.open(row['path']).convert('RGB')  # gri de olsa 3 kanala çevir
        x = self.tfms(img)
        y = torch.tensor([row['label_id']], dtype=torch.float32)
        return x, y

BATCH_SIZE = 16
NUM_WORKERS = 2
train_ds = MammoDataset(df_train, train_tfms)
val_ds   = MammoDataset(df_val,   val_tfms)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)


In [16]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", DEVICE)
# ---- 7) Model, Loss, Optimizer, Scheduler ----
MODEL_NAME = 'efficientnet_v2_s'
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
in_feats = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_feats, 1)  # binary logit
model.to(DEVICE)

pos_frac = max(1e-6, float(df_train['label_id'].mean()))
neg_frac = 1.0 - pos_frac
pos_weight = torch.tensor([neg_frac / pos_frac], device=DEVICE, dtype=torch.float32)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

LR = 3e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))

BEST_PATH = str(OUTPUT_DIR / 'best_cbis_ddsm.pt')


Device: cuda


/tmp/ipython-input-1940992020.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=='cuda'))


In [17]:
# ---- 8) Eğitim (AMP + early stopping + CSV log) ----
EPOCHS = 12
EARLY_STOP_PATIENCE = 4
best_auc = -1.0
epochs_no_improve = 0
history = []

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    for imgs, ys in train_loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        ys   = ys.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
            logits = model(imgs)
            loss = criterion(logits, ys)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        train_loss += loss.item() * imgs.size(0)
    scheduler.step()
    train_loss /= len(train_loader.dataset)

    # validation
    model.eval()
    val_loss = 0.0
    y_true, y_prob = [], []
    with torch.no_grad():
        for imgs, ys in val_loader:
            imgs = imgs.to(DEVICE, non_blocking=True)
            ys   = ys.to(DEVICE, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                logits = model(imgs)
                loss = criterion(logits, ys)
            val_loss += loss.item() * imgs.size(0)
            y_true.extend(ys.detach().cpu().numpy().ravel().tolist())
            y_prob.extend(torch.sigmoid(logits).detach().cpu().numpy().ravel().tolist())
    val_loss /= len(val_loader.dataset)
    try:
        val_auc = roc_auc_score(y_true, y_prob)
    except:
        val_auc = float('nan')
    y_pred = (np.array(y_prob) >= 0.5).astype(int)
    val_acc = accuracy_score(y_true, y_pred)
    val_f1  = f1_score(y_true, y_pred)

    print(f"[Epoch {epoch:02d}] train_loss={train_loss:.4f}  val_loss={val_loss:.4f}  AUC={val_auc:.4f}  ACC={val_acc:.4f}  F1={val_f1:.4f}")

    history.append({"epoch": epoch, "train_loss": float(train_loss), "val_loss": float(val_loss),
                    "val_auc": float(val_auc) if not np.isnan(val_auc) else None,
                    "val_acc": float(val_acc), "val_f1": float(val_f1),
                    "lr": float(scheduler.get_last_lr()[0])})

    if val_auc > best_auc:
        best_auc = val_auc; epochs_no_improve = 0
        torch.save({'model': model.state_dict(),
                    'model_name': MODEL_NAME,
                    'img_size': IMG_SIZE}, BEST_PATH)
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= EARLY_STOP_PATIENCE:
            print("Early stopping tetiklendi."); break

pd.DataFrame(history).to_csv(OUTPUT_DIR / 'train_log.csv', index=False)
print("En iyi AUC:", best_auc)
print("Model kaydedildi:", BEST_PATH)

/tmp/ipython-input-2539991457.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
/tmp/ipython-input-2539991457.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[Epoch 01] train_loss=0.4508  val_loss=0.4365  AUC=0.9445  ACC=0.8659  F1=0.8092


/tmp/ipython-input-2539991457.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
/tmp/ipython-input-2539991457.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[Epoch 02] train_loss=0.3984  val_loss=0.3729  AUC=0.9451  ACC=0.8659  F1=0.8068


/tmp/ipython-input-2539991457.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
/tmp/ipython-input-2539991457.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


...

[Epoch 10] train_loss=0.3166  val_loss=0.3543  AUC=0.9539  ACC=0.8469  F1=0.7995


/tmp/ipython-input-2539991457.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
/tmp/ipython-input-2539991457.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[Epoch 11] train_loss=0.3110  val_loss=0.3496  AUC=0.9540  ACC=0.8439  F1=0.7954


/tmp/ipython-input-2539991457.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
/tmp/ipython-input-2539991457.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


[Epoch 12] train_loss=0.3121  val_loss=0.3532  AUC=0.9541  ACC=0.8420  F1=0.7947
En iyi AUC: 0.9540882288358443
Model kaydedildi: /content/drive/MyDrive/breast_cancer/best_cbis_ddsm.pt


In [18]:
# ---- 9) En iyi modeli yükle & değerlendir ----
ckpt = torch.load(BEST_PATH, map_location=DEVICE)
model = efficientnet_v2_s(weights=None)
in_feats = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_feats, 1)
model.load_state_dict(ckpt['model']); model.to(DEVICE); model.eval()

y_true, y_prob = [], []
with torch.no_grad():
    for imgs, ys in val_loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        ys   = ys.to(DEVICE, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
            logits = model(imgs)
        y_true.extend(ys.detach().cpu().numpy().ravel().tolist())
        y_prob.extend(torch.sigmoid(logits).detach().cpu().numpy().ravel().tolist())
y_pred = (np.array(y_prob) >= 0.5).astype(int)

val_auc = float(roc_auc_score(y_true, y_prob))
val_acc = float(accuracy_score(y_true, y_pred))
val_f1  = float(f1_score(y_true, y_pred))
print("\nValidation AUC:", val_auc, "ACC:", val_acc, "F1:", val_f1)

# CM görseli ve rapor
fig, ax = plt.subplots(figsize=(4,4))
disp = ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred), display_labels=['benign','malignant'])
disp.plot(ax=ax, values_format='d', colorbar=False)
plt.title('Validation Confusion Matrix'); plt.tight_layout()
cm_path = OUTPUT_DIR / 'confusion_matrix.png'
plt.savefig(cm_path, dpi=150); plt.close(fig)

report_txt = classification_report(y_true, y_pred, target_names=['benign','malignant'])
with open(OUTPUT_DIR / 'classification_report.txt', 'w') as f: f.write(report_txt)

val_pred_df = pd.DataFrame({
    "path": [str(p) for p in df_val['path'].tolist()],
    "label": df_val['label'].tolist(),
    "prob_malignant": list(np.array(y_prob)),
    "pred_label": ['malignant' if i==1 else 'benign' for i in y_pred]
})
val_pred_df.to_csv(OUTPUT_DIR / 'val_predictions.csv', index=False)

metrics = {"val_auc": val_auc, "val_accuracy": val_acc, "val_f1": val_f1}
with open(OUTPUT_DIR / 'metrics.json', 'w') as f: json.dump(metrics, f, indent=2)

print("\nKaydedildi:")
for p in ["train_log.csv","confusion_matrix.png","classification_report.txt","val_predictions.csv","metrics.json","best_cbis_ddsm.pt"]:
    print(" -", OUTPUT_DIR / p)

/tmp/ipython-input-1754347368.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):



Validation AUC: 0.9540882288358443 ACC: 0.8419765166340509 F1: 0.7946598855689765

Kaydedildi:
 - /content/drive/MyDrive/breast_cancer/train_log.csv
 - /content/drive/MyDrive/breast_cancer/confusion_matrix.png
 - /content/drive/MyDrive/breast_cancer/classification_report.txt
 - /content/drive/MyDrive/breast_cancer/val_predictions.csv
 - /content/drive/MyDrive/breast_cancer/metrics.json
 - /content/drive/MyDrive/breast_cancer/best_cbis_ddsm.pt


In [19]:
# ---- 10) Basit TTA (flip) ----
def predict_with_tta(model, loader):
    probs = []
    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                p1 = torch.sigmoid(model(imgs)).detach().cpu().numpy().ravel()
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3]))).detach().cpu().numpy().ravel()
            probs.extend(((p1 + p2) / 2.0).tolist())
    return np.array(probs)

tta_prob = predict_with_tta(model, val_loader)
tta_auc = float(roc_auc_score(y_true, tta_prob))
with open(OUTPUT_DIR / 'metrics.json', 'r') as f: m = json.load(f)
m['tta_auc'] = tta_auc
with open(OUTPUT_DIR / 'metrics.json', 'w') as f: json.dump(m, f, indent=2)
print("TTA AUC:", tta_auc)

/tmp/ipython-input-3963247420.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):


TTA AUC: 0.9547951734960021


In [20]:
# ---- 11) Görsel özetler (metrik bar, ROC, PR, pie) ----
# 1) Metrik bar grafiği
metrics_plot = {"Accuracy": val_acc, "F1": val_f1, "AUC": val_auc}
if 'tta_auc' in m: metrics_plot['TTA AUC'] = float(m['tta_auc'])
plt.figure(figsize=(6,4))
plt.bar(list(metrics_plot.keys()), list(metrics_plot.values()))
plt.ylim(0, 1.0); plt.ylabel("Score"); plt.title("Validation Metrics")
for i,(k,v) in enumerate(metrics_plot.items()):
    plt.text(i, min(0.98, v)+0.02, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
plt.tight_layout(); plt.savefig(OUTPUT_DIR / 'metrics_bar.png', dpi=150); plt.close()

# 2) ROC
fpr, tpr, _ = roc_curve(y_true, y_prob)
auc_val = roc_auc_score(y_true, y_prob)
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC = {auc_val:.3f}")
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Validation)"); plt.legend(loc="lower right")
plt.tight_layout(); plt.savefig(OUTPUT_DIR / 'roc_curve.png', dpi=150); plt.close()

# 3) Precision-Recall
prec, rec, _ = precision_recall_curve(y_true, y_prob)
ap = average_precision_score(y_true, y_prob)
plt.figure(figsize=(5,5))
plt.plot(rec, prec, label=f"AP = {ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Validation)"); plt.legend(loc="lower left")
plt.tight_layout(); plt.savefig(OUTPUT_DIR / 'pr_curve.png', dpi=150); plt.close()

# 4) Tahmin edilen sınıf oranları
pred_counts = pd.Series(y_pred).value_counts()
benign_cnt = int(pred_counts.get(0, 0))
malig_cnt  = int(pred_counts.get(1, 0))
plt.figure(figsize=(5,5))
plt.pie([benign_cnt, malig_cnt], labels=["benign", "malignant"], autopct="%1.1f%%", startangle=90)
plt.title("Predicted Class Ratios (Validation)")
plt.tight_layout(); plt.savefig(OUTPUT_DIR / 'predicted_ratios_pie.png', dpi=150); plt.close()

# (Opsiyonel) Ground Truth oranları
gt_counts = pd.Series(np.array(y_true).astype(int)).value_counts()
benign_gt = int(gt_counts.get(0, 0))
malig_gt  = int(gt_counts.get(1, 0))
plt.figure(figsize=(5,5))
plt.pie([benign_gt, malig_gt], labels=["benign","malignant"], autopct="%1.1f%%", startangle=90)
plt.title("Ground Truth Class Ratios (Validation)")
plt.tight_layout(); plt.savefig(OUTPUT_DIR / 'ground_truth_ratios_pie.png', dpi=150); plt.close()

print("\nGörseller kaydedildi:")
for f in ["metrics_bar.png","roc_curve.png","pr_curve.png","predicted_ratios_pie.png","ground_truth_ratios_pie.png"]:
    print(" -", OUTPUT_DIR / f)

print("\nTamamlandı →", OUTPUT_DIR)


Görseller kaydedildi:
 - /content/drive/MyDrive/breast_cancer/metrics_bar.png
 - /content/drive/MyDrive/breast_cancer/roc_curve.png
 - /content/drive/MyDrive/breast_cancer/pr_curve.png
 - /content/drive/MyDrive/breast_cancer/predicted_ratios_pie.png
 - /content/drive/MyDrive/breast_cancer/ground_truth_ratios_pie.png

Tamamlandı → /content/drive/MyDrive/breast_cancer
